In [ ]:
import pandas as pd
import string
import operator
from pprint import pprint
import tqdm

In [ ]:
import gensim
from gensim.utils import simple_preprocess
from gensim import corpora, models
from gensim.models import CoherenceModel
import pyLDAvis.gensim
from gensim.models import CoherenceModel

In [ ]:
import nltk
from stop_words import get_stop_words
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords
import re

In [ ]:
import numpy as np
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score
import sklearn.metrics as metrics

In [ ]:
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble

from keras.preprocessing import text, sequence
from keras import layers, optimizers

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
stop_words = list(get_stop_words('en'))         #About 900 stopwords
#print(stop_words)
nltk_words = list(stopwords.words('english')) #About 150 stopwords
#print(nltk_words)
stop_words.extend(nltk_words)
#print(stop_words)

In [ ]:
Twitter = pd.read_csv("Academia.csv")
Twitter.head()

In [ ]:
#Twitter = Twitter.sample(n = 3000)
Twitter = Twitter.reset_index(drop = True)
Twitter.head()

In [ ]:
T_Text = Twitter["Text"]

In [ ]:
T_Text["Index"] = Twitter.set_index
T_Text = pd.DataFrame(T_Text)

In [ ]:
T_Text.head()

In [ ]:
T_Text = T_Text[T_Text['Text'].notnull()]
T_Text.loc[:,"Text"] = T_Text.Text.apply(lambda x : " ".join(re.findall('\w+',str(x)))) # remove punctuation
T_Text.loc[:,"Text"] = T_Text.Text.apply(lambda x : str.lower(x)) # lower case
T_Text["Text"] = T_Text.Text.str.replace('\d+', '') # remove numbers

In [ ]:
porter_stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer() 

In [ ]:
def remove_stopWords(s):
    s = ' '.join(word for word in s.split() if word not in stop_words)
    return s
T_Text.loc[:,"Text"] = T_Text.Text.apply(lambda x: remove_stopWords(x)) # stop words

In [ ]:
def stem_sentences(sentence):
    tokens = sentence.split()
    stemmed_tokens = [porter_stemmer.stem(token) for token in tokens]
    return ' '.join(stemmed_tokens)

T_Text.loc[:,"Text"] = T_Text.Text.apply(stem_sentences)

In [ ]:
def lemm_sentences(sentence):
    tokens = sentence.split()
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return lemmatized_tokens

T_Text.loc[:,"Text"] = T_Text.Text.apply(lemm_sentences)

In [ ]:
dictionary = gensim.corpora.Dictionary(T_Text["Text"])

In [ ]:
count = 0
for k, v in dictionary.items():
    print(k, v)
    count += 1
    if count > 10:
        break

In [ ]:
dictionary.filter_extremes(no_below=5, no_above=0.5, keep_n=1000000)

In [ ]:
# doc2bow
bow_corpus = [dictionary.doc2bow(doc) for doc in T_Text["Text"]]

# LDA using BOW Base Model

In [ ]:
lda_model = gensim.models.LdaMulticore(corpus=bow_corpus,
                                           id2word=dictionary,
                                           num_topics=10) 
                                           #random_state=100,
                                           #chunksize=100,
                                           #passes=10,
                                           #alpha=0.1,
                                           #eta=0.1)
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts = T_Text["Text"], dictionary = dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

In [ ]:
lda_display = pyLDAvis.gensim.prepare(lda_model, bow_corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display) # Graph


In [ ]:
def compute_coherence_values(corpus, dictionary, k, a, b):
    
    lda_model = gensim.models.LdaMulticore(corpus=bow_corpus,
                                           id2word=dictionary,
                                           num_topics=10, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=a,
                                           eta=b,
                                           per_word_topics=True)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=T_Text["Text"], dictionary=dictionary, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

In [ ]:
grid = {}
grid['Validation_Set'] = {}
# Topics range
min_topics = 5
max_topics = 16
step_size = 1
topics_range = range(min_topics, max_topics, step_size)
# Alpha parameter
alpha = list(np.arange(0.01, 1, 0.3))
alpha.append('symmetric')
alpha.append('asymmetric')
# Beta parameter
beta = list(np.arange(0.01, 1, 0.3))
beta.append('symmetric')
# Validation sets
num_of_docs = len(bow_corpus)
corpus_sets = [#gensim.utils.ClippedCorpus(corpus, num_of_docs*0.25), 
               #gensim.utils.ClippedCorpus(corpus, num_of_docs*0.5), 
               #gensim.utils.ClippedCorpus(bow_corpus, num_of_docs*0.75), 
               bow_corpus]
corpus_title = ['100% Corpus']
model_results = {'Validation_Set': [],
                 'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                }
# Can take a long time to run
if 1 == 1:
    pbar = tqdm.tqdm(total = 50)
    
    # iterate through validation corpuses
    for i in range(len(corpus_sets)):
        # iterate through number of topics
        for k in topics_range:
            # iterate through alpha values
            for a in alpha:
                # iterare through beta values
                for b in beta:
                    # get the coherence score for the given parameters
                    cv = compute_coherence_values(corpus=bow_corpus[i], dictionary=dictionary, 
                                                  k=k, a=a, b=b)
                    # Save the model results
                    model_results['Validation_Set'].append(corpus_title[i])
                    model_results['Topics'].append(k)
                    model_results['Alpha'].append(a)
                    model_results['Beta'].append(b)
                    model_results['Coherence'].append(cv)
                    
                    pbar.update(1)
    pd.DataFrame(model_results).to_csv('lda_tuning_results_Google.csv', index=False)
    pbar.close()

In general, higher alpha values mean documents contain more similar topic contents. The same is true for beta, but with topics and words: generally a high beta will result in topics with more similar word contents. Also, an asymmetric alpha is helpful, where as an asymmetric beta is largely not.

First we decide num_topics by controling alpha and beta.
Then we get our number of topics, then we decide alpha and beta to get our model. 

In [ ]:
lda_model = gensim.models.LdaMulticore(corpus=bow_corpus,
                                           id2word=dictionary,
                                           num_topics=8, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=0.01,
                                           eta=0.01)

In [ ]:
for idx, topic in lda_model.print_topics(-1, num_words=50):
    print('Topic: {} \nWords: {}'.format(idx, topic))

In [ ]:
def format_topics_sentences(ldamodel=lda_model, corpus=bow_corpus, texts=T_Text["Text"]):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[bow_corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Percentage Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=bow_corpus, texts=T_Text["Text"])


df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

